In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import SmoothBivariateSpline
from sklearn.impute import KNNImputer
from scipy.spatial import cKDTree
from scipy.interpolate import griddata

# Wind Data Upload + Interpolation

In [3]:
# Upload wind data
wind = pd.read_csv('windproducts.csv', dtype = 'string')
## time (UTC, string), altitude (m, string), latitude (degrees north, string), longitude (degrees east, string) 
### ekman_upwelling (m/s, string), wind_u (m/s, string), wind_v (m/s, string)

# Dytpe needs to be float for interpolation, but NaN's need to be removed first
df = pd.DataFrame(wind)
df = df.drop(0) # first row are units, no actual data
df_dropped_subset = df.dropna(subset=['ekman_upwelling', 'wind_u', 'wind_v'])

# Create new dataframe with necessary variables (with and without NaNs)
## these are used to loop interpolated values into df2
wind_df = df_dropped_subset[['latitude', 'longitude', 'ekman_upwelling', 'wind_u', 'wind_v']].astype(float) # creates dataframe without NaNs
df2 = df[['latitude', 'longitude', 'ekman_upwelling', 'wind_u', 'wind_v']].astype(float) # creates dataframe with interpolated values

# Normalize Data
df_mean = wind_df.mean()
df_stdev = wind_df.std()

wind_df_norm = (wind_df - df_mean) / df_stdev

df2_norm = (df2 - df_mean) / df_stdev

In [4]:
# Interpolate wind data using spline interpolation
## user warning indicates that interpolation was completed but may have had issues with convergence
### ordinary kriging may be prefered but due dataset size, kernel may crash and restart (ensure enough ram is available!)

# define variables for simplicity
## due to size of data and to avoid overestimation, sample and sample2 were defined 
### wind_u was overestimated with the sample parameters, however, wind_v became overestimated with sample2 parameters, ekman_upwelling appeared to be fine with either
#### using the normalized data also caused overestimation to occur so used non-normalized data to interpret u and v wind

sample = wind_df_norm.sample(n = 100, random_state = 50)
sample2 = wind_df_norm.sample(n = 1500, random_state = 15)
lon = sample['longitude']
lat = sample['latitude']
ekman = sample['ekman_upwelling']
wind_v = sample['wind_v']
wind_u = sample2['wind_u']
lon2 = sample2['longitude']
lat2 = sample2['latitude']

# ekman_upwelling interpolation
spline_ekman = SmoothBivariateSpline(lon, lat, ekman, s = 1)

mask_ekman_nan = df2['ekman_upwelling'].isna()
lon_nan = df2_norm.loc[mask_ekman_nan, 'longitude'].values
lat_nan = df2_norm.loc[mask_ekman_nan, 'latitude'].values
ekman_interp_norm = spline_ekman.ev(lon_nan, lat_nan)
ekman_interp = ekman_interp_norm * df_stdev['ekman_upwelling'] + df_mean['ekman_upwelling']
df2.loc[mask_ekman_nan, 'ekman_upwelling'] = ekman_interp

# wind_u (zonal, wind in the x-direction) interpolation
spline_u = SmoothBivariateSpline(lon2, lat2, wind_u, s = 0.0005)

mask_u_nan = df2['wind_u'].isna()
lon_nan_u = df2_norm.loc[mask_u_nan, 'longitude'].values
lat_nan_u = df2_norm.loc[mask_u_nan, 'latitude'].values
# u_interp_norm = spline_u.ev(lon_nan_u, lat_nan_u)
# u_interp = u_interp_norm * df_stdev['wind_u'] + df_mean['wind_u']
df2.loc[mask_u_nan, 'wind_u'] = spline_u.ev(lon_nan_u, lat_nan_u)

# wind_v (meridonal, wind in the y-direction) interpolation
spline_v = SmoothBivariateSpline(lon, lat, wind_v, s = 0.05)

mask_v_nan = df2['wind_v'].isna()
lon_nan_v = df2_norm.loc[mask_v_nan, 'longitude'].values
lat_nan_v = df2_norm.loc[mask_v_nan, 'latitude'].values
# v_interp_norm = spline_v.ev(lon_nan_v, lat_nan_v)
# v_interp = v_interp_norm * df_stdev['wind_v'] + df_mean['wind_v']
df2.loc[mask_v_nan, 'wind_v'] = spline_v.ev(lon_nan_v, lat_nan_v)
# df2.to_csv('check.csv', index = False) # to check interpolated values
## may need to restart kernel to get .csv file to show updated results

# u and v wind were extremely overestimated, some values were good, but some were 500+ m/s
# brining in more samples caused wind_v to overestimate, but wind u was more representative 

/var/folders/00/6s7jmy896l9gc4jq1245rx680000gn/T/ipykernel_1587/2060939219.py:21: UserWarning: ier=988
  spline_ekman = SmoothBivariateSpline(lon, lat, ekman, s = 1)
/var/folders/00/6s7jmy896l9gc4jq1245rx680000gn/T/ipykernel_1587/2060939219.py:31: UserWarning: ier=1299
  spline_u = SmoothBivariateSpline(lon2, lat2, wind_u, s = 0.0005)
/var/folders/00/6s7jmy896l9gc4jq1245rx680000gn/T/ipykernel_1587/2060939219.py:41: UserWarning: ier=1299
  spline_v = SmoothBivariateSpline(lon, lat, wind_v, s = 0.05)


In [5]:
# Can use this to check values (used to determine why values were getting overestimated)
## currently set to check wind v
### print("Normalized values (min, max):", v_interp_norm.min(), v_interp_norm.max())
### print("Unnormalized values (min, max):", v_interp.min(), v_interp.max())
### print("Mean:", df_mean['wind_v'], "| Std:", df_stdev['wind_v'])
### results showcase there are outliers present

In [6]:
# Date needs to be added back to dataset, but with time removed
## Original time format (string) caused errors
df['date'] = pd.to_datetime(df['time']).dt.date
date_float = (pd.to_datetime(df['date']) - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
date_string = df['date'].astype(str)

df2.insert(0, 'date (float)', date_float) # only needs to be done once, will get error if ran twice in the same session
df2.insert(1, 'date (string)', date_string) # only needs to be done once, will get error if ran twice in the same session

df2[['latitude','longitude', 'wind_u', 'wind_v']] = df2[['latitude','longitude', 'wind_u', 'wind_v']].round(3) # to ensure lon and lat for both datasets are similar

# Sea-Surface Temperature Data Upload + Interpolation

In [8]:
# Upload SST data
sst = pd.read_csv('sst.csv', dtype = 'string')
# time (UTC, string), zlev (m, string), latitude (degrees north, string), longitude (degrees east, string), sst (degrees C, string)

# dytpe needs to be float for interpolation, but NaN's need to be removed first
df_ = pd.DataFrame(sst)
df_ = df_.drop(0)  # first row is units, not actual data
df_dropped_subset2 = df_.dropna(subset = ['sst'])

sst_df = df_dropped_subset2[['latitude', 'longitude', 'sst']].astype(float)
df3 = df_[['latitude', 'longitude', 'sst']].astype(float)

# Normalize the data
df_mean2 = sst_df.mean()
df_stdev2 = sst_df.std()

sst_df_norm = (sst_df - df_mean2) / df_stdev2

df3_norm = (df3 - df_mean2) / df_stdev2

In [9]:
# Interpolate sst data using spline interpolation
## user warning indicates that interpolation was completed but may have had issues with convergence

# Define variables for simplicity
sample = sst_df_norm.sample(n = 1000, random_state = 50)
lon = sample['longitude']
lat = sample['latitude']
sst_ = sample['sst']

# SST interpolation
spline_sst = SmoothBivariateSpline(lon, lat, sst_, s = 1)

mask_sst_nan = df3['sst'].isna()
lon_nan = df3_norm.loc[mask_sst_nan, 'longitude'].values
lat_nan = df3_norm.loc[mask_sst_nan, 'latitude'].values
sst_interp_norm = spline_sst.ev(lon_nan, lat_nan)
sst_interp = mask_sst_nan * df_stdev2['sst'] + df_mean2['sst']
df3.loc[mask_sst_nan, 'sst'] = sst_interp
# df3.to_csv('checkcheck.csv', index = False) # to check interpolated values
# no apparent issues with overestimation

/var/folders/00/6s7jmy896l9gc4jq1245rx680000gn/T/ipykernel_1587/221099362.py:11: UserWarning: ier=2116
  spline_sst = SmoothBivariateSpline(lon, lat, sst_, s = 1)


In [10]:
# Date needs to be added back to dataset, but with time removed
## Original time format caused errors
df_['date'] = pd.to_datetime(df_['time']).dt.date
date_float = (pd.to_datetime(df_['date']) - pd.Timestamp('1970-01-01')) // pd.Timedelta('1D')
date_string = df_['date'].astype(str)

df3.insert(0, 'date (float)', date_float) # only needs to be done once, will get error if ran twice in the same session
df3.insert(1, 'date (string)', date_string) # only needs to be done once, will get error if ran twice in the same session

df3[['latitude','longitude', 'sst']] = df3[['latitude','longitude', 'sst']].round(3)

# Merge Wind and SST Datasets + Generate New CSV File

In [12]:
# Merge the wind and sst datasets 
## df3 (sst) is being merged to df2 (wind)
merged_data = [] # creates a dataframe where the merged datasets will be stored

for date in df2['date (string)'].dropna().unique(): # checks unique dates and extracts wind and sst data
    wind_day = df2[df2['date (string)'] == date]
    sst_day = df3[df3['date (string)'] == date]
    merged = pd.merge(wind_day, sst_day[['latitude', 'longitude', 'sst']], on = ['latitude', 'longitude'], how = 'left')
    missing_mask = merged['sst'].isna() # checks to see if there are any sst values missing after merge

    if missing_mask.any() and not sst_day.empty: # sst and wind have diff lengths, this fills in those NaNs
        known_points = sst_day[['latitude', 'longitude']].values
        known_values = sst_day['sst'].astype(float).values
        missing_points = merged.loc[missing_mask, ['latitude', 'longitude']].values
        interpolated = griddata(known_points, known_values, missing_points, method = 'linear')
        if np.any(np.isnan(interpolated)):
            interpolated_nn = griddata(known_points, known_values, missing_points, method = 'nearest')
            interpolated = np.where(np.isnan(interpolated), interpolated_nn, interpolated)
        merged.loc[missing_mask, 'sst'] = interpolated
        
    merged_data.append(merged) # puts everything into the empty merged_data dataframe

# Combine all merged days into a single DataFrame
merged_data = pd.concat(merged_data, ignore_index=True)
merged_data['date (string)'] = pd.to_datetime(merged_data['date (string)'], format='%Y-%m-%d')
final_df = merged_data[merged_data['date (string)'] <= '2025-03-29']
#final_df.to_csv('wind_sst.csv', index = False)

# Defining Upwelling Parameters + Generating New CSV File
##### For Upwelling to occur in the California Current System
##### - Duration: >12 hours or more of specific wind direction and speed (for this dataset, 1+ days and 3+ day)*
##### - Magnitude: >5 m/s 
##### - Direction: Northwesterly / Equatorward (wind_v < 0, wind_u > 0)**
######
###### *upwelling duration is typically 1-2 weeks, chose 3+ days to determine upwelling event to avoid overestimation
###### **wind_v and wind_u are the meridonal and zonal winds, primarily indicates direction so calculated wind_speed

In [14]:
# Upload data 
upwelling = pd.read_csv('wind_sst.csv')
upwelling_df = pd.DataFrame(upwelling)

In [15]:
# Define variables
wind_u = upwelling_df['wind_u']
wind_v = upwelling_df['wind_v']
sst = upwelling_df['sst']
date = pd.to_datetime(upwelling_df['date (string)'])

# Create wind speed variable using u and v wind
upwelling_df['wind_speed'] = np.sqrt(upwelling_df['wind_u']**2 + upwelling_df['wind_v']**2) # need for wind magnitude
wind_speed = upwelling_df['wind_speed']

In [16]:
# Defining a sst decrease
## code from non-spatial section was used to elaborate for spatial data
### code groups all data by unique lat and lon pair, and sorts via the date, then makes notes if SST dropped the day before or not (T or F)
#### sst drops in response to upwelling bring colder bottom waters to the surface, acts more of a check than a parameter for upwelling
upwelling_df = upwelling_df.sort_values(['latitude', 'longitude', 'date (string)']).copy()

# Compute SST change at each location over time
upwelling_df['sst_diff'] = (upwelling_df.groupby(['latitude', 'longitude'])['sst'].diff())
upwelling_df['sst_decrease'] = upwelling_df['sst_diff'] < 0

In [17]:
# Defining upwelling based on wind speed, u wind, v wind and sst based on 1 day
## CCS upwelling periods are 1-2 weeks, but upwelling events can also occur for longer periods
def upwelling(wind_u, wind_v, wind_speed, sst_decrease):    
    if (wind_speed > 5 and wind_u > 0 and wind_v < 0 and sst_decrease == True):
        return 1
    else:
        return 0

# Create new column upwelling1day in upwelling_df
## lambda row essentially loops through each row to see if parameters as defined in upwelling is met
upwelling_df['upwelling1day'] = upwelling_df.apply(lambda row: upwelling(row['wind_u'], row['wind_v'], row['wind_speed'], row['sst_decrease']), axis=1)

In [18]:
# Defining upwelling based on wind speed, u wind, v wind and sst based on 3 days or more of parameters being met
upwelling_df = upwelling_df.sort_values(['latitude', 'longitude', 'date (string)']).copy()

def upwelling3day(group):
    group_id = (group['upwelling1day'] != group['upwelling1day'].shift()).cumsum()
    group_sizes = group.groupby(group_id)['upwelling1day'].transform('sum')
    group['upwelling3day'] = np.where((group['upwelling1day'] == 1) & (group_sizes >= 3),1,0 )
    return group

# Function is applied to each coordinate group
upwelling_df = upwelling_df.groupby(['latitude', 'longitude'], group_keys = False).apply(upwelling3day)

/var/folders/00/6s7jmy896l9gc4jq1245rx680000gn/T/ipykernel_1587/3321504030.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  upwelling_df = upwelling_df.groupby(['latitude', 'longitude'], group_keys = False).apply(upwelling3day)


## Defining Upwelling Non-Spatially

In [20]:
# Create new csv file
upwelling_df1 = upwelling_df.round({'wind_u':4, 'wind_v':3, 'wind_speed':3, 'sst':2,'wind_speed':3})
#upwelling_df1.to_csv('upwelling_spatial.csv', index = False)

In [21]:
# To determine which lon and lat pairs appear in each day of the dataset
#total_dates = upwelling_df['date (float)'].nunique()
#pair_counts = (upwelling_df.groupby(['latitude', 'longitude'])['date (float)'].nunique().reset_index(name='date_count'))
#consistent_pairs = pair_counts[pair_counts['date_count'] == total_dates][['latitude', 'longitude']]
#consistent_pairs

In [22]:
# Create new non-spatial dataset
upwelling1 = pd.read_csv('wind_sst.csv')
upwelling_df1 = pd.DataFrame(upwelling1)

lat = 34.833
lon = -123.833
df_filtered = upwelling_df1[(upwelling_df1['latitude'] == lat) & (upwelling_df1['longitude'] == lon)].copy()
df_filtered = df_filtered.dropna()
df_filtered = df_filtered.iloc[1:].reset_index(drop=True) # dropped first row as it goes from 2021-09-03 to 2021-09-08

In [23]:
# Define variables
wind_u = df_filtered['wind_u']
wind_v = df_filtered['wind_v']
sst = df_filtered['sst']

# Create wind speed variable from u and v wind
df_filtered['wind_speed'] = np.sqrt(df_filtered['wind_u']**2 + df_filtered['wind_v']**2) # need for wind magnitude
wind_speed = df_filtered['wind_speed']

In [24]:
# Create new column that determines if sst drops (T or F)
df_filtered['sst_diff'] = df_filtered['sst'].diff()
df_filtered['sst_decrease'] = df_filtered['sst_diff'] < 0 

In [25]:
# Defining upwelling based on wind speed, u wind, v wind and sst based on 1 day
def upwelling3(wind_u, wind_v, wind_speed, sst_decrease):    
    if (wind_speed > 5 and wind_u > 0 and wind_v < 0 and sst_decrease == True):
        return 1
    else:
        return 0

df_filtered['upwelling1day'] = df_filtered.apply(lambda row: upwelling3(row['wind_u'], row['wind_v'], row['wind_speed'], row['sst_decrease']), axis=1)

In [26]:
# Defining upwelling based on wind speed, u wind, v wind and sst based on 3 days or more of parameters being met
df_filtered = df_filtered.sort_values('date (string)').reset_index(drop = True) # sorts by day
group_id = (df_filtered['upwelling1day'] != df_filtered['upwelling1day'].shift()).cumsum() # creates an id variable to find consecutive days 
group_sizes = df_filtered.groupby(group_id)['upwelling1day'].transform('sum') # determines the length of consecutive days
df_filtered['upwelling3day'] = np.where((df_filtered['upwelling1day'] == 1) & (group_sizes >= 3),1,0) # creates new column that only includes when parameters are met for 3+ days

In [27]:
# Create new csv file
#df_filtered.to_csv('upwelling_nonspatial.csv', index = False)